<a href="https://colab.research.google.com/github/cris-her/People_search/blob/master/People_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IA: algoritmos evolutivos, lógica difusa, agentes, sistemas expertos, machine learning.

ML: (dataset... clasificar o predecir)

supervisado: [scikit-learn] regresion lineal[p], regresion logistica[c], naive bayes[c], arbol de decisiones[p, c], random forest[p, c], k-nearest nighbours[p, c], dl. 10k

no supervisado: k-means, mean-shift [clasifican].

reforzado.

DeepLearning: [pytorch, tensorflow-keras] redes neuronales. 1M
DeepNeuralNetwork, RecurrentNN(tiempo-audio), ConvolutionalNN(imagenes-GenerativeAdversarialNetwork).

Nvidia - NeuralModules - Modelo


* reconocimiento: voz-[texto] descripcion --- biometricos suaves [diccionario]

http://www.sc.ehu.es/ccwbayes/docencia/mmcc/docs/t8neuronales.pdf

1.   Voz a texto --preentrenado
2.   Dataset - etiquetado [entrenamiento-pruebas] preprocessing..posible 30240
3.   DNN(modelo) - preentrenado(fine tuning) -epochs -pesos sinapticos
sobreentrenado - subentrenado
hiperparametros - parametros
4.   Inferencia - prueba

4 partes: ____

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.0.0rc1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

In [3]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
audio, sample_rate = librosa.load('description.wav')

ipd.Audio('description.wav', rate=sample_rate)

In [4]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-02-26 01:35:37 experimental:28] Module <class 'nemo.collections.asr.models.clustering_diarizer.ClusteringDiarizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...


[NeMo W 2021-02-26 01:35:38 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data]   Unzipping corpora/cmudict.zip.


In [5]:
# This line will download pre-trained QuartzNet15x5 model from NVIDIA's NGC cloud and instantiate it for you
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2021-02-26 01:35:38 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-02-26 01:35:39 common:615] Instantiating model from pre-trained checkpoint
[NeMo I 2021-02-26 01:35:40 features:239] PADDING: 16
[NeMo I 2021-02-26 01:35:40 features:255] STFT using torch
[NeMo I 2021-02-26 01:35:51 modelPT:413] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.


In [6]:
files = ['description.wav']
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  print(f"Audio in {fname} was recognized as: {transcription}")

[NeMo W 2021-02-26 01:35:51 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-02-26 01:35:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
      normalized, onesided, return_complex)
    



Audio in description.wav was recognized as: a young white boy with black hair wearing a gray jacket


In [7]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

In [8]:
# set the following paths
DATA_DIR = "PATH_TO_DATA"
WORK_DIR = "PATH_TO_CHECKPOINTS_AND_LOGS"

In [9]:
## download get_squad.py script to download and preprocess the SQuAD data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_squad.py'):
    print('Downloading get_squad.py...')
    wget.download(f'https://raw.githubusercontent.com/cris-her/People_search/master/get_squad.py', WORK_DIR)
else:
    print ('get_squad.py already exists')

In [9]:
# download and preprocess the data
! python $WORK_DIR/get_squad.py --destDir $DATA_DIR

In [9]:
! ls -LR {DATA_DIR}/squad

In [9]:
# This is the model configuration file that we will download, do not change this
MODEL_CONFIG = "question_answering_squad_config.yaml"

# model parameters, play with these
BATCH_SIZE = 12
MAX_SEQ_LENGTH = 384
# specify BERT-like model, you want to use
PRETRAINED_BERT_MODEL = "bert-base-uncased"
TOKENIZER_NAME = "bert-base-uncased" # tokenizer name

# Number of data examples used for training, validation, test and inference
TRAIN_NUM_SAMPLES = VAL_NUM_SAMPLES = TEST_NUM_SAMPLES = 5000 
INFER_NUM_SAMPLES = 5

TRAIN_FILE = f"{DATA_DIR}/squad/v1.1/train-v1.1.json"
VAL_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"
TEST_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"
INFER_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"

INFER_PREDICTION_OUTPUT_FILE = "output_prediction.json"
INFER_NBEST_OUTPUT_FILE = "output_nbest.json"

# training parameters
LEARNING_RATE = 0.00003

# number of epochs
MAX_EPOCHS = 1